In [1]:
import sys

# Add the parent directory to the system path
sys.path.append("../04_survival_models/src")

In [2]:
import re
from collections import Counter

import mlflow
import pandas as pd
import seaborn as sns
import numpy as np
from azureml.core import Workspace
from uc2_functions import *
from tqdm import tqdm

In [3]:
sns.set(style="whitegrid")

# Goal

The goal is to collect results from Monte Carlo simulations.

# Parameters

In [4]:
# Legend
PATH_LEGEND = "Legenda_Variabili_Uri_Larcher.xlsx"
# Type of dataset
DATASET = "raw"  # raw, larcher
EXPERIMENT_NAME = (
    "UC2_raw_2024_02" if DATASET == "raw" else "UC2_larcher_2024_02"
)
# Metrics
PATH_METRICS = "df_metrics_{}.csv".format(EXPERIMENT_NAME)
# Feature importance
PATH_IMPORTANCES = "df_importances_{}.csv".format(EXPERIMENT_NAME)
# Directories
DIR_SC = os.path.join(os.path.dirname(os.getcwd()), "sc")  # Legend
DIR_ARTIFACTS = "artifacts"
# Number of simulations
S = 100
# Number of models
N_MODELS = 3

# Data ingestion

## One-hot encoding version

In [5]:
if DATASET == "raw":
    # azureml-core of version 1.0.72 or higher is required
    # azureml-dataprep[pandas] of version 1.1.34 or higher is required

    subscription_id = "753a0b42-95dc-4871-b53e-160ceb0e6bc1"
    resource_group = "rg-s-race-aml-dev-we"
    workspace_name = "amlsraceamldevwe01"

    workspace = Workspace(subscription_id, resource_group, workspace_name)

    dataset = Dataset.get_by_name(workspace, name="UC2_raw_survival_csm_ohe_5yrs")
    df_ohe = dataset.to_pandas_dataframe()
    print(df_ohe.shape)
    df_ohe.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
(2536, 211)


### Use schema

Recreate the schema from tags:

In [6]:
if DATASET == "raw":
    tags = dataset.tags

    dtypes = json.loads(tags["dtypes_json"])
    is_ordinal = json.loads(tags["is_ordinal_json"])

    for col in dtypes.keys():
        if dtypes[col] == "category":
            categories = (
                sorted(df_ohe[col].dropna().unique())
                if is_ordinal[col]
                else df_ohe[col].dropna().unique()
            )
            df_ohe[col] = pd.Categorical(
                df_ohe[col], categories=categories, ordered=is_ordinal[col]
            )
        else:
            df_ohe[col] = df_ohe[col].astype(dtypes[col])

    count_columns_by_dtype(df_ohe)

float64: 31
boolean: 171
ordinal category: 9
non ordinal category: 0


## .xlsx Legend

In [7]:
if DATASET == "raw":
    df_legend = pd.read_excel(
        os.path.join(DIR_SC, PATH_LEGEND), sheet_name="Variabili Etichette DBURI"
    )
    dict_legend = pd.Series(
        df_legend["Etichetta"].values, index=df_legend["Variabile"]
    ).to_dict()

# Get from mlflow

In [8]:
workspace = Workspace.from_config()
experiment = workspace.experiments[EXPERIMENT_NAME]
# Set the MLflow tracking URI to point to your Azure ML workspace
mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())
client = mlflow.tracking.MlflowClient()

In [9]:
df_metrics = collect_simulations(experiment=experiment,
                                 client=client,
                                 dir_artifacts=DIR_ARTIFACTS)
print(df_metrics.shape)
df_metrics = df_metrics.groupby('random_state').filter(lambda x: len(x) == N_MODELS).reset_index(drop=True)
print(df_metrics.shape)
df_metrics.to_csv(os.path.join(DIR_ARTIFACTS, PATH_METRICS), index=False)

0it [00:00, ?it/s]

1it [00:02,  2.66s/it]

4it [00:04,  1.28it/s]

5it [00:05,  1.15it/s]

8it [00:07,  1.64it/s]

...

393it [04:06,  1.62it/s]

396it [04:08,  2.09it/s]

397it [04:09,  1.59it/s]

400it [04:11,  1.59it/s]

300
(300, 11)
(300, 11)


# Feature importance

## T1

In [10]:
if DATASET == "raw":
    timepoint = "T1"
    len_most_common = 25

    pattern = "df_importance_rsf_" + timepoint
    l = []
    for f in [filename for filename in os.listdir(DIR_ARTIFACTS) if pattern in filename]:
        l.append(pd.read_json(os.path.join(DIR_ARTIFACTS, f)).iloc[:25].index.tolist())
    dataset = "Raw DBURI dataset" if DATASET == "raw" else "Clinician’s dataset"
    df_importances_t1 = count_occurrences(l, len_most_common=len_most_common)
    df_importances_t1["Dataset"] = dataset
    df_importances_t1["Model"] = "Random Survival Forest – T1"
    df_importances_t1["Monte-Carlo simulations"] = len(l)
    df_importances_t1["Feature compact"] = df_importances_t1["Feature"]
    df_importances_t1["Feature extended"] = df_importances_t1["Feature compact"].apply(
        lambda x: replace_longest_match(x, dict_legend)
    )
    df_importances_t1 = df_importances_t1[
        [
            "Dataset",
            "Model",
            "Monte-Carlo simulations",
            "Feature compact",
            "Feature extended",
            "Freq. top 25",
            "Freq. top 5",
            "Freq. first rank",
        ]
    ]
    df_importances_t1.sort_values(["Freq. top {}".format(len_most_common), "Freq. top 5", "Freq. first rank"], ascending=[False, False, False], inplace=True)
    del l

## T0

In [11]:
if DATASET == "raw":
    timepoint = "T0"


    pattern = "df_importance_rsf_" + timepoint
    l = []
    for f in [filename for filename in os.listdir(DIR_ARTIFACTS) if pattern in filename]:
        l.append(pd.read_json(os.path.join(DIR_ARTIFACTS, f)).iloc[:25].index.tolist())
    dataset = "Raw DBURI dataset" if DATASET == "raw" else "Clinician’s dataset"
    df_importances_t0 = count_occurrences(l, len_most_common=25)
    df_importances_t0["Dataset"] = dataset
    df_importances_t0["Model"] = "Random Survival Forest – T0"
    df_importances_t0["Monte-Carlo simulations"] = len(l)
    df_importances_t0["Feature compact"] = df_importances_t0["Feature"]
    df_importances_t0["Feature extended"] = df_importances_t0["Feature compact"].apply(
        lambda x: replace_longest_match(x, dict_legend)
    )
    df_importances_t0 = df_importances_t0[
        [
            "Dataset",
            "Model",
            "Monte-Carlo simulations",
            "Feature compact",
            "Feature extended",
            "Freq. top 25",
            "Freq. top 5",
            "Freq. first rank",
        ]
    ]
    df_importances_t0.sort_values(["Freq. top {}".format(len_most_common), "Freq. top 5", "Freq. first rank"], ascending=[False, False, False], inplace=True)
    del l

## Concat

In [12]:
if DATASET == "raw":
    df_importances = pd.concat([df_importances_t1, df_importances_t0])
    del df_importances_t1, df_importances_t0

## Get dtypes

In [13]:
if DATASET == "raw":
    df_importances['Type'] = df_importances['Feature compact'].apply(lambda x: df_ohe.dtypes.to_dict()[x])

## Export

In [14]:
if DATASET == "raw":
    df_importances.to_csv(os.path.join(DIR_ARTIFACTS, PATH_IMPORTANCES), index=False)